# svg_dataset

> End-to-end functions taking in centerline-stroke SVG's and outputting deltas in Stroke-3 format.

In [ ]:
#| default_exp stroke3

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# |hide

In [ ]:
#| export
from singleline_dataset.path_helpers import discretize_paths
from singleline_dataset.path_transforms import rdpify, rescale_strokes, transform_paths
from singleline_dataset.stroke3 import strokes_to_deltas
from singleline_dataset.svg_files import load_svg
from singleline_dataset.svg_transforms import svg_to_transforms


def svg_to_stroke_deltas(
    input_fname, total_n=1000, min_n=3, target_size=200, epsilon=2.0
):
    paths, attributes, svg_attributes, svg_root = load_svg(input_fname)
    all_strokes = discretize_paths(paths, total_n=total_n, min_n=min_n)

    globally_rescaled_strokes = transform_paths(
        all_strokes, global_transform=svg_to_transforms(svg_root)
    )

    rescaled_strokes = rescale_strokes(globally_rescaled_strokes, target_size)
    rdp_strokes = rdpify(rescaled_strokes, epsilon)
    deltas = strokes_to_deltas(rdp_strokes)

    ## roundtrip / sanity check
    # _rdp_strokes = stroke3.deltas_to_strokes(deltas)
    # display_plot.plot_strokes(_rdp_strokes)

    return deltas, rescaled_strokes

ModuleNotFoundError: No module named 'singleline_dataset.svg_global'

In [ ]:
#| export
def points_to_deltas(points):
    p2 = points.copy()
    # first row should stay the same
    # cols 0,1 of every row onwards should be a delta from the previous row.
    p2[1:, :2] = points[1:, :2] - points[:-1, :2]
    return p2

In [ ]:
#| export
def strokes_to_deltas(strokes):
    points = strokes_to_points(strokes)
    return points_to_deltas(points)

In [ ]:
#| export
def deltas_to_points(_seq):
    seq = np.zeros_like(_seq)
    seq[:, 0:2] = np.cumsum(_seq[:, 0:2], axis=0)
    seq[:, 2] = _seq[:, 2]
    return seq

In [ ]:
#| export
def points_to_strokes(_seq):
    strokes = np.split(_seq, np.where(_seq[:, 2] > 0)[0] + 1)
    return strokes

In [ ]:
#| export
def deltas_to_strokes(_seq):
    points = deltas_to_points(_seq)
    return points_to_strokes(points)

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()